### Data processor and shaper of Energy Balance Matrix.

> This module integrates the Latin America and Caribbean Energy Balance Matrix and sort out its data in a OSeMOSYS model structure. According to the naming convention used by OSeMOSYS a 3-letter country abbreviation will be adopted and added at the tail of the code (label) as follows:

    [category][sector][fuel/commodity][###][region]

e.g. for "Crude oil extraction" kind of technology in "Argentina" the generic convention label is: `MINCRUARG`

---
Author: Mario R. Peralta. A. <br>
School of Electrical Engineering, University of Costa Rica. <br>
The Electric Power and Energy Research Laboratory (EPERLab)

---




Example of implementation the `resLAC` module: Filtering technologies per regions.

In [1]:
import resLAC
from resLAC import Primary_Tech
from resLAC import Secondary_Tech

directory = "./data/matrix.xlsx"
matrix = resLAC.EnergyMatrix()
matrix.build_RES(directory)


In [2]:
# Filter primary Costa Rican technology
primary_tech_cri = [T for T in matrix.technologies
                    if (isinstance(T, Primary_Tech)
                        and "CRI" in T.code)]
print(primary_tech_cri)

[MINCRUCRI, IMPCRUCRI, EXPCRUCRI, WASTECCRUCRI, MINNGSCRI, IMPNGSCRI, EXPNGSCRI, WASTECNGSCRI, MINCOA001CRI, IMPCOA001CRI, EXPCOA001CRI, WASTECCOA001CRI, MINHYDCRI, IMPHYDCRI, EXPHYDCRI, WASTECHYDCRI, MINGEOCRI, IMPGEOCRI, EXPGEOCRI, WASTECGEOCRI, MINNUCCRI, IMPNUCCRI, EXPNUCCRI, WASTECNUCCRI, MINWOOCRI, IMPWOOCRI, EXPWOOCRI, WASTECWOOCRI, MINSGCCRI, IMPSGCCRI, EXPSGCCRI, WASTECSGCCRI, MINOPRCRI, IMPOPRCRI, EXPOPRCRI, WASTECOPRCRI, PWRELCCRI, IMPELCCRI, EXPELCCRI, WASTECELCCRI, MINOHCCRI, IMPOHCCRI, EXPOHCCRI, WASTECOHCCRI, MINGSLCRI, IMPGSLCRI, EXPGSLCRI, WASTECGSLCRI, MINKERCRI, IMPKERCRI, EXPKERCRI, WASTECKERCRI, MINDSLCRI, IMPDSLCRI, EXPDSLCRI, WASTECDSLCRI, MINHFOCRI, IMPHFOCRI, EXPHFOCRI, WASTECHFOCRI, MINCOKCRI, IMPCOKCRI, EXPCOKCRI, WASTECCOKCRI, MINCOA002CRI, IMPCOA002CRI, EXPCOA002CRI, WASTECCOA002CRI, MINGASCRI, IMPGASCRI, EXPGASCRI, WASTECGASCRI, MINOSECRI, IMPOSECRI, EXPOSECRI, WASTECOSECRI, MINNENCRI, IMPNENCRI, EXPNENCRI, WASTECNENCRI]


In [3]:
# Filter secondary Brazil technology
second_tech_bra = [T for T in matrix.technologies
                   if (isinstance(T, Secondary_Tech)
                       and "BRA" in T.code)]
print(second_tech_bra)

[UPSREFCRUBRA, UPSPWRCRUBRA, UPSSELCRUBRA, UPSGASCRUBRA, UPSCOACRUBRA, UPSBOICRUBRA, UPSDETCRUBRA, UPSTECCRUBRA, DEMTRACRUBRA, DEMINDCRUBRA, DEMRESCRUBRA, DEMCOMCRUBRA, DEMAGRCRUBRA, DEMCONCRUBRA, DEMNEECRUBRA, UPSREFNGSBRA, UPSPWRNGSBRA, UPSSELNGSBRA, UPSGASNGSBRA, UPSCOANGSBRA, UPSBOINGSBRA, UPSDETNGSBRA, UPSTECNGSBRA, DEMTRANGSBRA, DEMINDNGSBRA, DEMRESNGSBRA, DEMCOMNGSBRA, DEMAGRNGSBRA, DEMCONNGSBRA, DEMNEENGSBRA, UPSREFCOA001BRA, UPSPWRCOA001BRA, UPSSELCOA001BRA, UPSGASCOA001BRA, UPSCOACOA001BRA, UPSBOICOA001BRA, UPSDETCOA001BRA, UPSTECCOA001BRA, DEMTRACOA001BRA, DEMINDCOA001BRA, DEMRESCOA001BRA, DEMCOMCOA001BRA, DEMAGRCOA001BRA, DEMCONCOA001BRA, DEMNEECOA001BRA, UPSREFHYDBRA, UPSPWRHYDBRA, UPSSELHYDBRA, UPSGASHYDBRA, UPSCOAHYDBRA, UPSBOIHYDBRA, UPSDETHYDBRA, UPSTECHYDBRA, DEMTRAHYDBRA, DEMINDHYDBRA, DEMRESHYDBRA, DEMCOMHYDBRA, DEMAGRHYDBRA, DEMCONHYDBRA, DEMNEEHYDBRA, UPSREFGEOBRA, UPSPWRGEOBRA, UPSSELGEOBRA, UPSGASGEOBRA, UPSCOAGEOBRA, UPSBOIGEOBRA, UPSDETGEOBRA, UPSTECGEOBRA, DE

In [4]:
print(second_tech_bra[0].input_commodity.energy_PJ)
print(second_tech_bra[0].output_commodity.energy_PJ)


-3872.365111277474
3872.365111277474


In [3]:
# Filter all technology of Argentina
tech_arg = [T for T in matrix.technologies if ("ARG" in T.code)]
print(tech_arg)

[MINCRUARG, IMPCRUARG, EXPCRUARG, WASTECCRUARG, UPSREFCRUARG, UPSPWRCRUARG, UPSSELCRUARG, UPSGASCRUARG, UPSCOACRUARG, UPSBOICRUARG, UPSDETCRUARG, UPSTECCRUARG, DEMTRACRUARG, DEMINDCRUARG, DEMRESCRUARG, DEMCOMCRUARG, DEMAGRCRUARG, DEMCONCRUARG, DEMNEECRUARG, MINNGSARG, IMPNGSARG, EXPNGSARG, WASTECNGSARG, UPSREFNGSARG, UPSPWRNGSARG, UPSSELNGSARG, UPSGASNGSARG, UPSCOANGSARG, UPSBOINGSARG, UPSDETNGSARG, UPSTECNGSARG, DEMTRANGSARG, DEMINDNGSARG, DEMRESNGSARG, DEMCOMNGSARG, DEMAGRNGSARG, DEMCONNGSARG, DEMNEENGSARG, MINCOA001ARG, IMPCOA001ARG, EXPCOA001ARG, WASTECCOA001ARG, UPSREFCOA001ARG, UPSPWRCOA001ARG, UPSSELCOA001ARG, UPSGASCOA001ARG, UPSCOACOA001ARG, UPSBOICOA001ARG, UPSDETCOA001ARG, UPSTECCOA001ARG, DEMTRACOA001ARG, DEMINDCOA001ARG, DEMRESCOA001ARG, DEMCOMCOA001ARG, DEMAGRCOA001ARG, DEMCONCOA001ARG, DEMNEECOA001ARG, MINHYDARG, IMPHYDARG, EXPHYDARG, WASTECHYDARG, UPSREFHYDARG, UPSPWRHYDARG, UPSSELHYDARG, UPSGASHYDARG, UPSCOAHYDARG, UPSBOIHYDARG, UPSDETHYDARG, UPSTECHYDARG, DEMTRAHYDARG

In [7]:
# All waste technology of Argentina
waste_tech_arg = [W for W in tech_arg if "WASTEC" in W.code]
print(waste_tech_arg)

[WASTECCRUARG, WASTECNGSARG, WASTECCOA001ARG, WASTECHYDARG, WASTECGEOARG, WASTECNUCARG, WASTECWOOARG, WASTECSGCARG, WASTECOPRARG, WASTECELCARG, WASTECOHCARG, WASTECGSLARG, WASTECKERARG, WASTECDSLARG, WASTECHFOARG, WASTECCOKARG, WASTECCOA002ARG, WASTECGASARG, WASTECOSEARG, WASTECNENARG]


Natural gas waste technology of argentina: WASTECNGSARG

In [9]:
# WASTECNGSARG energy
waste_tech_arg[1].output_commodity.energy_PJ

333.9870134943714